# Preparing the Base Model

In [1]:
import os

In [2]:
%pwd

'c:\\Users\\harik\\Desktop\\All Folder\\PGDA\\Interships & Projects\\NLP\\End-to-End Workflow\\cnnClassifier\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\harik\\Desktop\\All Folder\\PGDA\\Interships & Projects\\NLP\\End-to-End Workflow\\cnnClassifier'

In [5]:
# Now below we are writting the entity

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen= True)
class PrepareBaseModelconfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [7]:
# Now after updating entity we need to update configuration_manager

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> PrepareBaseModelconfig:
          
          config = self.config.prepare_base_model

          create_directories([config.root_dir])

          prepare_base_model_config = PrepareBaseModelconfig(
                root_dir= Path(config.root_dir),
                base_model_path= Path(config.base_model_path),
                updated_base_model_path= Path(config.updated_base_model_path),
                params_image_size = self.params.IMAGE_SIZE,
                params_learning_rate = self.params.LEARNING_RATE,
                params_include_top = self.params.INCLUDE_TOP,
                params_weights = self.params.WEIGHTS,
                params_classes = self.params.CLASSES
          )

          return prepare_base_model_config

In [10]:
# Now we write the components

In [11]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [25]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelconfig):

        self.config = config

    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape = self.config.params_image_size,
            weights= self.config.params_weights,
            include_top= self.config.params_include_top
        )

        self.save_model(path= self.config.base_model_path, model=self.model)   # Save_model defined at last


    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):

        if freeze_all:
            for layer in model.layers:
                model.trainable = False                          # Freezing all the layers

        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)    

        """ 
            We need to do the flattem explicitely becuase we are only taking the convolution part
            of the model with include_top = False, thus it wont have its predetermined fully connected layers
            so we add the flatten layer here 
         
         """

        prediction = tf.keras.layers.Dense(
            units= classes,
            activation= 'softmax'
        )(flatten_in)

        """ 
            Now we created a Dense layer, This is the output layer, here we are not creating any hidden layyers for the learning purpose,
            as the we are using the defualt weights from imagenet, which is a huge dataset, so our model is already trained for indentifying
            various objects using imagenet dataset. So here we are only using the convolutional layer of the VGG16 model as a feature 
            extractor. [ Here even if we retrain, we may not need a lot of computational resources as the weights are already computed ].
            SO we are only using a ouput layer with 2 neurons.

        """


        full_model = tf.keras.models.Model(
            inputs= model.input,
            outputs= prediction
        )

        full_model.compile(
            optimizer= tf.keras.optimizers.SGD(learning_rate= learning_rate),
            loss= tf.keras.losses.CategoricalCrossentropy(),
            metrics= ["accuracy"]
            )
        
        full_model.summary()
        return full_model


    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model= self.model,
            classes= self.config.params_classes,
            learning_rate= self.config.params_learning_rate,
            freeze_all= True,
            freeze_till= None
        )

        self.save_model(path= self.config.updated_base_model_path, model= self.full_model)


    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [16]:
# Below we create the pipeline

In [26]:
try:
    config= ConfigurationManager()
    prepare_base_model_config= config.get_prepare_base_model_config()
    prepare_base_model= PrepareBaseModel(config= prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()

except Exception as e:
    raise e

[2023-10-01 14:15:22,524: INFO: common: yaml file: config\config.yaml loaded succesfully]
[2023-10-01 14:15:22,537: INFO: common: yaml file: params.yaml loaded succesfully]
[2023-10-01 14:15:22,545: INFO: common: created directory at: artifacts]
[2023-10-01 14:15:22,552: INFO: common: created directory at: artifacts/prepare_base_model]


58889256/58889256 [==============================] - 18s 0us/step
[2023-10-01 14:15:42,906: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     738

c:\Users\harik\anaconda3\envs\cnncls\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       295168    
                                                                 
 block3_conv2 (Conv2D)       (None, 56, 56, 256)       590080    
                                                                 
 block3_conv3 (Conv2D)       (None, 56, 56, 256)       590080    
                                                                 
 block3_pool (MaxPooling2D)  (None, 28, 28, 256)       0         
                                                                 
 block4_conv1 (Conv2D)       (None, 28, 28, 512)       1180160   
                                                                 
 block4_conv2 (Conv2D)       (None, 28, 28, 512)       2359808   
                                                                 
 block4_co